# Prompt Engineering with Snowflake Cortex

This notebook demonstrates how to leverage Snowflake's `AI_COMPLETE` function to interact with large language models (LLMs).  
Using well‑crafted prompts is critical for obtaining accurate and useful responses. The examples below illustrate best practices for prompt engineering—including clear instructions, structured output, chain‑of‑thought reasoning, few‑shot examples, role prompting, and tuning model parameters.

> **Note:** The queries shown here are examples. You should modify the model names and parameters based on the models available in your Snowflake environment.  For deterministic outputs, lower the temperature; for more creative outputs, increase it. Always test prompts with different settings and review the outputs before using them in production.


## Best Practices for Prompt Engineering

Effective prompts guide the model to produce reliable, structured responses. Consider these general guidelines when designing prompts:

- **Be clear and specific:** Describe the task explicitly, including necessary context and the desired format.  If the task involves multiple steps, enumerate them.
- **Repeat critical instructions:** Reinforce important constraints at the end of the prompt—LLMs often prioritise the most recent instructions.
- **Use structural markers:** Separate sections of your prompt with separators (e.g. `---`) or tags such as `<instructions>`, `<thinking>`, and `<answer>` to clearly delineate content, reasoning, and final answers.
- **Provide examples:** Show a few diverse examples of inputs and desired outputs so the model learns the pattern (few‑shot prompting).  Wrap examples in tags like `<example>` to help the model parse them.
- **Encourage step‑by‑step thinking:** For complex reasoning, instruct the model to think through the problem step by step or include chain‑of‑thought tags.  This often improves correctness.
- **Specify the output structure:** Ask for specific formats (e.g. JSON) and, when available, use the `response_format` argument to enforce a JSON schema.  Structured outputs reduce parsing logic in SQL and improve consistency.
- **Tune model parameters:** Adjust `temperature` and `top_p` to control randomness.  A low `temperature` (e.g. 0–0.3) yields deterministic responses; higher values (0.7–1.0) encourage diversity.  Change one parameter at a time and evaluate the results.


## 1. Simple Prompt

A simple prompt asks a single, well‑defined question.  Use clear language and avoid ambiguity.  When you only need a short answer and do not require a specific format, default parameters often suffice.


In [ ]:
-- Ask a direct question using the default model
SELECT AI_COMPLETE(
  'llama3.2-3b',
  'What are large language models and why are they important in modern AI?'  
);


## 2. LLM as a Judge (Self‑Evaluation)

You can chain calls to `AI_COMPLETE` to have one model answer a question and a second model evaluate that answer.  Use this pattern to rate responses, classify them, or check policy compliance.  The grading prompt should instruct the evaluator to return a structured result (e.g. a JSON object with a score).  Parsing the output with `PARSE_JSON` makes it easy to extract fields.

Below, we ask a model to answer a simple question and then ask a different model to score the answer based on length.


In [ ]:
-- Define the question
SET question_text = 'How many words are in your answer?';

WITH qa AS (
  SELECT
    $question_text AS question,
    AI_COMPLETE(
      model  => 'llama3.2-3b',
      prompt => $question_text
    ) AS answer
),
graded AS (
  SELECT
    question,
    answer,
    AI_COMPLETE(
      model => 'claude-3-7-sonnet',
      prompt => 'Score the following answer on a scale of 1 to 10 for answering the question completely.

Question: ' || question || '
Answer: ' || answer,
      model_parameters => { 'temperature': 0 },
      response_format => {
        'type': 'json',
        'schema': {
          'type': 'object',
          'properties': {
            'score': { 'type': 'integer' }
          }
        }
      }
    ) AS evaluation_json
  FROM qa
)
SELECT 
  question, 
  answer, 
  evaluation_json:score::INT AS score
FROM graded;

## 3. Using Tags and Chain‑of‑Thought Reasoning

For tasks that require reasoning, you can instruct the model to expose its thinking process inside specific tags like `<thinking>` or `<analysis>`.  At the end of the prompt, repeat the desired output structure to reinforce it.  When evaluating the answer, focus on the content in the `<answer>` tags.

The example below asks the model to calculate which company retreat option fits a budget.  We instruct it to show intermediate calculations inside `<thinking>` tags and provide a final recommendation outside the tags.


In [ ]:
-- Chain of thought with custom tags
SELECT AI_COMPLETE(
  'claude-4-sonnet',
  $$A tech startup has grown from 45 employees to 180 employees this year! They want to celebrate with a company retreat. They have a budget of $125,000 and are considering two options:

' ||
  'Option A: 3‑day retreat at a mountain resort ($650 per person for 3 days, includes meals)
' ||
  'Option B: 2‑day retreat at a beach resort ($420 per person for 2 days) + $200 per person bonus

' ||
  'Which option fits within their budget?

' ||
  'Think through the calculations inside <thinking> tags, then provide your final recommendation in <answer> tags.

' ||
  'Format:
<thinking>...step‑by‑step calculations...</thinking>
<answer>...your recommendation...</answer>$$
);


## 4. Structured Output with JSON Schema

When you need to extract information reliably from model outputs, ask the model to respond in JSON and use the `response_format` argument.  Define a JSON schema with the expected properties and data types.  Setting a low `temperature` improves determinism.  Snowflake will validate the response against the schema, ensuring that the model outputs valid JSON.

In this example, we summarise a product review and classify its sentiment.  The `response_format` enforces a JSON object with two properties—`summary` (a string) and `sentiment` (one of `positive`, `negative`, or `neutral`).


In [ ]:
SELECT AI_COMPLETE(
  model => 'claude-4-sonnet',
  prompt => 'Summarise the following product review in one sentence and classify its overall sentiment as positive, negative, or neutral:
"I''m very happy with the battery life but the screen is too dim."',
  response_format => {
    'type': 'json',
    'schema': {
      'type': 'object',
      'properties': {
        'summary': {'type': 'string'},
        'sentiment': {'type': 'string', 'enum': ['positive','negative','neutral']}
      },
      'required': ['summary','sentiment']
    }
  },
  model_parameters => { 'temperature': 0 }
);

## 5. Tuning Model Parameters

`temperature` and `top_p` control the randomness of the output.  Lower values (e.g. `temperature = 0.0`) yield more deterministic answers; higher values (e.g. `temperature = 0.8`) encourage diverse phrasing.  Tune one parameter at a time.  You can also set `max_tokens` to limit the length of the output.

Below we generate customer service responses to the same message using two different temperatures.  Notice how the tone changes.


In [ ]:
-- Example messages
WITH support_tickets AS (
  SELECT * FROM VALUES
    ('High priority billing issue needs resolution'),
    ('Customer confused about new feature'),
    ('Product recommendation request for enterprise client')
  AS t(customer_message)
)
SELECT 
  customer_message,
  -- Creative response with higher temperature
  AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Write a friendly and informal customer service reply to: ' || customer_message,
    model_parameters => { 'temperature': 0.8,'top_p': 0.9}
  ) AS creative_response,
  -- Focused response with lower temperature
  AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Write a professional customer service reply to: ' || customer_message,
    model_parameters => { 'temperature': 0.2, 'top_p': 0.1 }
  ) AS professional_response
FROM support_tickets;

## 6. Few‑Shot Prompting with Examples

Providing a few examples helps the model learn the structure of the desired output.  Encapsulate examples inside `<examples>` and `<example>` tags to separate them from the main instructions.  Make sure your examples are diverse and representative of real inputs.

The next query extracts key fields from support tickets.  It includes two annotated examples of `Input` and `Output` pairs.  The model is instructed to return **only** valid JSON with fields `input`, `issue`, `urgency`, `company`, and `user_id`.


In [ ]:
WITH ticket_analysis AS (
  SELECT AI_COMPLETE(
    'claude-4-sonnet',
    $$Extract key information from customer support tickets and output ONLY valid JSON with fields "input", "issue", "urgency", "company", and "user_id".

<examples>
<example>
Input: "The login page won't load on Chrome. Been trying for 20 minutes. UserID: jsmith_99"
Output: {"input": "The login page won't load on Chrome. Been trying for 20 minutes. UserID: jsmith_99", "issue": "Login failure", "urgency": "medium", "company": null, "user_id": "jsmith_99"}
</example>

<example>
Input: "Our API returns a 500 error when calling /invoices. Need a fix ASAP. Company: FinTechCo"
Output: {"input": "Our API returns a 500 error when calling /invoices. Need a fix ASAP. Company: FinTechCo", "issue": "API error", "urgency": "high", "company": "FinTechCo", "user_id": null}
</example>
</examples>

Extract key information from this ticket. Return ONLY valid JSON with no other text:
"Hey! The dashboard keeps crashing when I try to export my Q3 sales data. This is super urgent because I have a board meeting tomorrow at 9am. My company is TechCorp and my user ID is tc_sarah_m. Please help!!!"$$
  ) AS extraction
)
SELECT extraction
FROM ticket_analysis;

### Flatten the JSON

In [ ]:
WITH ticket_analysis AS (
  SELECT AI_COMPLETE(
    'claude-4-sonnet',
    $$Extract key information from customer support tickets and output ONLY valid JSON with fields "input", "issue", "urgency", "company", and "user_id".

Input: "The login page won't load on Chrome. Been trying for 20 minutes. UserID: jsmith_99"
Output: {"input": "The login page won't load on Chrome. Been trying for 20 minutes. UserID: jsmith_99", "issue": "Login failure", "urgency": "medium", "company": null, "user_id": "jsmith_99"}

Input: "Our API returns a 500 error when calling /invoices. Need a fix ASAP. Company: FinTechCo"
Output: {"input": "Our API returns a 500 error when calling /invoices. Need a fix ASAP. Company: FinTechCo", "issue": "API error", "urgency": "high", "company": "FinTechCo", "user_id": null}

Extract key information from this ticket. Return ONLY valid JSON with no other text: "Hey! The dashboard keeps crashing when I try to export my Q3 sales data. This is super urgent because I have a board meeting tomorrow at 9am. My company is TechCorp and my user ID is tc_sarah_m. Please help!!!"$$
  ) AS extraction
),
parsed_data AS (
  SELECT PARSE_JSON(extraction) AS json_data
  FROM ticket_analysis
)
SELECT 
  json_data:input::STRING AS ticket_input,
  json_data:issue::STRING AS issue_description,
  json_data:urgency::STRING AS urgency_level,
  json_data:company::STRING AS company_name,
  json_data:user_id::STRING AS user_id
FROM parsed_data;

## 7. Chain‑of‑Thought Reasoning for Business Decisions

Complex decisions benefit from step‑by‑step analysis.  Instruct the model to think through the metrics sequentially inside a `<thinking>` tag and to summarise its recommendation in a separate `<recommendation>` tag.  Listing the steps in the prompt helps the model structure its reasoning and ensures that all relevant factors are considered.


In [ ]:
-- Step‑by‑step reasoning with enumerated steps and tags
SELECT AI_COMPLETE(
  'claude-4-sonnet',
  $$A SaaS company has these monthly metrics:
' ||
  '- Monthly Recurring Revenue (MRR): $125,000
' ||
  '- Customer Acquisition Cost (CAC): $450 per customer
' ||
  '- Average Revenue Per User (ARPU): $89/month
' ||
  '- Monthly Churn Rate: 3.2%
' ||
  '- Gross Margin: 78%

' ||
  'Should they increase their marketing spend by 40% to accelerate growth?

' ||
  'Think through this step‑by‑step in <thinking> tags:
' ||
  '1. Calculate the current customer base and acquisition metrics.
' ||
  '2. Determine customer lifetime value (LTV).
' ||
  '3. Analyze the LTV/CAC ratio and payback period.
' ||
  '4. Consider the impact of a 40% increase in marketing spend.
' ||
  '5. Evaluate whether the unit economics support increased spending.

' ||
  'Then provide your recommendation in <recommendation> tags.

' ||
  'Format:
' ||
  '<thinking>...your detailed analysis and calculations...</thinking>
' ||
  '<recommendation>...clear yes/no recommendation with key reasoning...</recommendation>$$
);


## 8. Role Prompting with Multiple Messages

Add roles to `AI_COMPLETE`, such as `system`, `user`, and `assistant`.  Role prompting helps set the persona or tone of the assistant.  Use a `system` message to define behaviour, a `user` message for the query, and optional `assistant` messages to provide context or examples.

In this example, the `system` message is instructing the model to act as a concise technical assistant.  The `user` message contains the actual question.


In [ ]:
-- Role prompting using a single string with role instructions
SELECT AI_COMPLETE(
  model => 'claude-4-sonnet',
  prompt => 'You are a helpful technical assistant who provides concise explanations.

User: Explain the difference between synchronous and asynchronous processing in plain language.',
  model_parameters => { 'temperature': 0.3 }
);

## 9. Parameter Controls

You can manage the model configurations within the call to help control for your responses.


In [ ]:
SELECT AI_COMPLETE(
  model => 'llama3.2-3b',
  prompt => 'Write a short 4-line poem about Snowflake. Use exactly 4 lines and end with <END>.

Poem:',
  model_parameters => {
    'max_tokens': 80,
    'temperature': 0.7,
    'top_p': 0.8,
    'guardrails': TRUE
  }
);

## 10. Classification, Filtering and Similarity

Snowflake Cortex includes additional AI SQL functions beyond text generation.  
`AI_CLASSIFY` assigns labels to text from a list of categories, `AI_FILTER` returns a boolean indicating whether text matches a natural‑language filter, and `AI_SIMILARITY` computes the similarity between two embeddings.  These functions let you build pipelines that classify content, filter records, or rank results before passing them to `AI_COMPLETE`.


### AI_CLASSIFY

In [ ]:
-- Enhanced sentiment classification with descriptions
WITH messages AS (
  SELECT * FROM VALUES
    ('Thank you for the great support!'),
    ('This product is terrible and doesn''t work'),
    ('The product is okay, nothing special')
  AS t(message)
)
SELECT 
  message,
  AI_CLASSIFY(
    message,
    ARRAY_CONSTRUCT(
      OBJECT_CONSTRUCT('label', 'Positive', 'description', 'expressing satisfaction, happiness, or approval'),
      OBJECT_CONSTRUCT('label', 'Negative', 'description', 'expressing dissatisfaction, anger, or disapproval'), 
      OBJECT_CONSTRUCT('label', 'Neutral', 'description', 'neither positive nor negative, factual or indifferent')
    ),
    OBJECT_CONSTRUCT(
      'task_description', 'Classify customer feedback sentiment for support analysis'
    )
  ) AS sentiment_result
FROM messages;

In [ ]:
-- Multi-label topic classification
WITH customer_feedback AS (
  SELECT * FROM VALUES
    ('The shipping was fast but the product quality is poor'),
    ('Great customer service and easy returns policy'),
    ('Expensive pricing but excellent build quality')
  AS t(feedback)
)
SELECT 
  feedback,
  AI_CLASSIFY(
    feedback,
    ARRAY_CONSTRUCT('shipping', 'quality', 'pricing', 'customer_service', 'returns'),
    OBJECT_CONSTRUCT(
      'output_mode', 'multi',
      'task_description', 'Identify all topics mentioned in customer feedback'
    )
  ) AS topics
FROM customer_feedback;

### AI_FILTER

In [ ]:
-- Filter records using PROMPT function
WITH products AS (
  SELECT * FROM VALUES
    ('A big orange cat painting'),
    ('Elegant blue evening dress'),
    ('Fast red sports car')
  AS t(description)
)
SELECT description
FROM products
WHERE AI_FILTER(PROMPT('Is this item related to fashion: {0}', description));

### AI_SIMILARITY

In [ ]:
-- Compute semantic similarity between sentences
WITH sentences AS (
  SELECT * FROM VALUES
    ('Large language models are transforming business operations.'),
    ('The weather this weekend will be sunny.'),
    ('LLMs can write code and analyse text.')
  AS t(sentence)
)
SELECT 
  sentence,
  AI_SIMILARITY(
    sentence,
    'Explain how LLMs impact businesses.'
  ) AS similarity_score
FROM sentences
ORDER BY similarity_score DESC;

## 11. Guardrails and Data Privacy

Enterprise applications must protect sensitive information and guard against inappropriate content.  Generative models can fabricate facts or leak data【180164900185176†L133-L139】.  Implementing guardrails involves:

- **Filtering prompts and outputs:** Use `AI_FILTER` or `AI_CLASSIFY` to detect sensitive content before passing it to an LLM or returning it to users.
- **Redacting personal data:** Preprocess input to remove or anonymize personally identifiable information (PII).
- **Human review:** Incorporate humans in the loop to audit critical outputs and handle exceptions【180164900185176†L228-L250】.

The SQL below illustrates how to screen text for potential sensitive content using `AI_FILTER`.


In [ ]:
-- Detect messages that may contain sensitive information
WITH messages AS (
  SELECT * FROM VALUES
    ('My credit card number is 4111 1111 1111 1111'),
    ('Looking forward to the meeting tomorrow!')
  AS t(message)
)
SELECT 
  message,
  AI_FILTER(PROMPT('Does this text contain sensitive personal or financial information: {0}', message)) AS contains_sensitive_data
FROM messages;